# Import Libraries

In [ ]:
!pip install sentinelhub==3.6.1
import datetime
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import os
from zipfile import ZipFile

!pip install pykml
!pip install fiona
!pip install geopandas

from pykml import parser
!pip install geojson
!pip install rasterio
import geojson
import geopandas as gpd
import fiona

from osgeo import gdal
import json
import re
import datetime
import os
from itertools import product
import rasterio as rio
from rasterio import windows
from tqdm import tqdm

fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw' # enable KML support which is disabled by default

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00
  Created wheel for sentinelhub: filename=sentinelhub-3.6.1-py3-none-any.whl size=231320 sha256=ba3679c9eb1ef3706ad508bf75ad688db276921e5ce2d9ffc60e6033c10386e4
  Stored in directory: /root/.cache/pip/wheels/af/d2/46/5635d21e3660be87a599b26b96e9c1bffb29aec23f8a213188
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6085 sha256=c62c060ad17699e45d264c5ee5de63b1e17639c49341c2dbdecf759ce9473e79
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built sentinelhub utm
     ━━━━━━━━━━━━━━━━━━━━━━━━

## Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Sentinel Hub API credentials

In [ ]:
from sentinelhub import SentinelHubRequest, DataCollection, MimeType, CRS, BBox, SHConfig, Geometry, MosaickingOrder, SentinelHubDownloadClient, bbox_to_dimensions, bbox_to_resolution, SentinelHubStatistical, SentinelHubStatisticalDownloadClient
# Credentials
config = SHConfig()
config.sh_client_id = '2880617b-83d6-4de8-a837-f7464b698235'
config.sh_client_secret = '1S:V#sa3{6I_/vhW.c(/K!A:0,)(2{:i<U<O*J0#'

## Specify Time Interval Function

In [ ]:
def wheat_time_interval(start_year, start_month, start_date, end_year, end_month, end_date, chunks):
  start = datetime.datetime(start_year, start_month, start_date)
  end = datetime.datetime(end_year, end_month , end_date)
  n_chunks = chunks
  tdelta = (end - start) / n_chunks
  edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
  slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

  print("Monthly time windows:\n")
  for slot in slots:
      print(slot)
  return slots

## Specify Evalscript for band acquisition


The evalscripts are for acquiring: <br>


1.   All bands of Sentinel2
2.   3 bands (RGB)
3.   1 band NDVI
4.   1 band DataMask

Comment/Uncomment the code as per requirement



In [ ]:

#*******************ALL BANDS************************************

evalscript = """

//VERSION=3


function setup() {
  return {
    input: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B11","B12", "dataMask"],
    output: { bands: 13, sampleType: "FLOAT32"}

  };
}

function evaluatePixel(sample) {
  return [sample.B01,
          sample.B02,
          sample.B03,
          sample.B04,
          sample.B05,
          sample.B06,
          sample.B07,
          sample.B08,
          sample.B8A,
          sample.B09,
          sample.B11,
          sample.B12,
          sample.dataMask];
}

"""

#*******************ALL BANDS************************************


# *******************3 BANDS RGB************************************
# evalscript = """
# //VERSION=3
# function setup(){
#   return{

#      input: [{
#       bands: ["B02", "B03", "B04"]
#     }],
#     output: {bands: 3 }
#   }
# }

# function evaluatePixel(sample){

#   return [sample.B04, sample.B03 , sample.B02];
# }
# """
#*******************3 BANDS RGB************************************


########################  1 BAND NDVI EVALSCRIPT ###############################################
# evalscript = """

# //VERSION=3
# function setup() {
#   return {
#     input: ["B04", "B08", "dataMask"],
#     output: { bands: 1, sampleType: "FLOAT32" }
#   };
# }

# function evaluatePixel(sample) {
#   if (sample.dataMask == 1) {
#     let ndvi = index(sample.B08, sample.B04);
#     return [ndvi];
#   } else {
#     return [99]
#   }
# }

# """

########################  1 BAND NDVI EVALSCRIPT ###############################################


# ########################  1 BAND DATA MASK EVALSCRIPT ###############################################

# evalscript = """

# //VERSION=3
# function setup() {
#   return {
#     input: ["dataMask"],
#     output: { bands: 1, sampleType: "FLOAT32" }
#   };
# }

# function evaluatePixel(sample) {
#   if (sample.dataMask == 1) {

#     return [1];
#   } else {
#     return [0]
#   }
# }

# """
########################  1 BAND DATA MASK EVALSCRIPT ###############################################


 ## GET TRUE COLOR

Satellite data can be downlaoded for a arbitraty polygon using 'Geometry' and for a rectangle boundary box using 'BBox' using code below:

 ### Geometry

Sentinel2 L2A data is acquired in this function using an arbitrary polygon/geometry. The output image returned is of size 48x48 with bicubuc upsampling and mosacking order of least cloud cover.<br>
The output size needs to be defined for the resulting file.

In [ ]:
def get_true_color_request(time_interval, file, Geometry1, path):
  return SentinelHubRequest(
  evalscript=evalscript,
  data_folder = os.path.join(path, '%s' %file),
  input_data=[
      SentinelHubRequest.input_data(
          data_collection=DataCollection.SENTINEL2_L2A,
          time_interval=time_interval,
          other_args={"dataFilter": {"maxCloudCoverage": 0, "mosaickingOrder": "leastCC"}, "processing": {"upsampling": "BICUBIC"}}
      ),
  ],
  responses=[
      SentinelHubRequest.output_response('default', MimeType.TIFF),
  ],
  geometry=Geometry1,
  size=[48, 48],
  config=config
)

 ### BBox

Sentinel2 L2A data is acquired in this function using a boundary box. The size of the output image is automaticaaly computed based on the passed resoltion of 10m or 60m etc. Bicubuc upsampling and mosacking order of least cloud cover is applied.

In [ ]:

def get_true_color_request_bbox(time_interval, file, bbox_coordinates, path, img_height,
                                img_width, resolution):

  bbox_size =  bbox_to_dimensions(bbox_coordinates, resolution=resolution)
  print("Image size is", bbox_size, "pixels")
  return SentinelHubRequest(
  evalscript=evalscript,
  data_folder = os.path.join(path, '%s' %file),
  input_data=[
      SentinelHubRequest.input_data(
          data_collection=DataCollection.SENTINEL2_L2A,
          time_interval=time_interval,
          other_args={"dataFilter": {"maxCloudCoverage": 0, "mosaickingOrder": "leastCC"},
                      "processing": {"upsampling": "BICUBIC"}}

      ),
  ],
  responses=[
      SentinelHubRequest.output_response('default', MimeType.TIFF),
  ],

  bbox=bbox_coordinates,
  size=bbox_size,
  config=config
)

##Create Data Request

In [ ]:
# create a list of requests
def get_sentinel_data(slots):
  list_of_requests = [get_true_color_request(slot) for slot in slots]
  # list_of_requests = [request.download_list[0] for request in list_of_requests]

  # download data with multiple threads
  data = SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=5)
  return data

##PRINT DATA

In [ ]:
# some stuff for pretty plots
def print_time_series(passed_data):
  ncols = 15
  nrows = 13
  aspect_ratio = 1
  subplot_kw = {"xticks": [], "yticks": [], "frame_on": False}

  fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5 * ncols * aspect_ratio, 5 * nrows), subplot_kw=subplot_kw)

  for idx, image in enumerate(passed_data):
      ax = axs[idx // ncols][idx % ncols]
      ax.imshow(np.clip(image * 2.5 / 255, 0, 1))
      ax.set_title(f"{slots[idx][0]}  -  {slots[idx][1]}", fontsize=10)
  plt.tight_layout()
  # plt.savefig('%s %d %s' %("Wheat", year, "Original"))

#Request data for BBox

##Convert to BBox with 2 corners only

In [ ]:
#function to get the 2 corners of a rectangle/square because BBox requires 2 corner coordinates only (top-left and bottom-right)
def bbox(coord_list):
     box = []
     for i in (0,1):
         res = sorted(coord_list, key=lambda x:x[i])
         box.append((res[0][i],res[-1][i]))
     box_list = [box[0][0], box[1][0], box[0][1], box[1][1]]
    #  ret = f"{box[0][0]}, {box[1][0]}, {box[0][1]}, {box[1][1]}"
     return box_list

#https://gis.stackexchange.com/questions/313011/convert-geojson-object-to-bounding-box-using-python-3-6

In [ ]:
# Create a new dataframe for storing the polygons
ground_truth = gpd.GeoDataFrame()
driver = 'KML'

#path to your .kml file
polygon_file_path = "/content/Untitled Polygon.kml"
#extract filename from directory
directory, file = os.path.split(polygon_file_path)

# polpulate the dataframe using the .kml file containing all polygons
for layer in fiona.listlayers(polygon_file_path):
  s = gpd.read_file(polygon_file_path, driver=driver, layer=layer)
  ground_truth = pd.concat([ground_truth, s], ignore_index=True)
  print(ground_truth)

##extract geometrical coordinates of all the polygons in the .kml file
multi_polygon_coordinates=[]
for geo_shape in ground_truth.geometry.values:
  temp_polygon = list(map(list, (geo_shape.exterior.coords)))
  # print("temp_polygon", temp_polygon)
  multi_polygon_coordinates.append(list([temp_polygon]))
print(multi_polygon_coordinates)

# parse your json here
poly=geojson.Polygon(multi_polygon_coordinates)
bbox_coordinates = bbox(list(geojson.utils.coords(poly))) #function call to get 2 corner coordinates of BBox
print(bbox_coordinates)

#specify resolution and coordinates in CRS
resolution = 10 #in meters
img_height = img_width=128
bbox_coordinates = BBox(bbox=bbox_coordinates, crs=CRS.WGS84)

##specify time slots
slots = wheat_time_interval(2021, 11, 1, 2022, 5, 17, 7) #start_year, start_month, start_date, end_year, end_month, end_date, chunks
results = [slot[1] for slot in slots]
print(results)

#specify path for saving data in drive
path = '/content/4/'
##get and save data
for slot in slots:
  data = get_true_color_request_bbox(slot, file, bbox_coordinates, path, img_height, img_width, resolution)
  data.save_data()


               Name Description  \
0  Untitled Polygon               

                                            geometry  
0  POLYGON Z ((74.08510 32.04691 0.00000, 74.1102...  
[[[[74.08509621709523, 32.04690553751833, 0.0], [74.11029447946628, 32.0469351520649, 0.0], [74.11036035817298, 32.06877269941585, 0.0], [74.08504604567621, 32.06880147017053, 0.0], [74.08509621709523, 32.04690553751833, 0.0]]]]
[74.085046, 32.046906, 74.11036, 32.068801]
Monthly time windows:

('2021-11-01', '2021-11-29')
('2021-11-29', '2021-12-27')
('2021-12-27', '2022-01-24')
('2022-01-24', '2022-02-21')
('2022-02-21', '2022-03-21')
('2022-03-21', '2022-04-18')
['2021-11-29', '2021-12-27', '2022-01-24', '2022-02-21', '2022-03-21', '2022-04-18']
Image size is (241, 241) pixels
Image size is (241, 241) pixels
Image size is (241, 241) pixels
Image size is (241, 241) pixels
Image size is (241, 241) pixels
Image size is (241, 241) pixels


##BBox to dimensions
*SPECIFY* resolution of 1 pixel and *GET* image size accordingly (in pixels)
E.g. Specifying 10m resolution gives (137, 79) pixels image size for whatever area (on ground in km2 etc.) you input

In [ ]:
resolution=20
betsiboka_size = bbox_to_dimensions(bbox_coordinates, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")

# https://sentinelhub-py.readthedocs.io/en/latest/examples/process_request.html

# bbox_to_dimensions
# """Calculates width and height in pixels for a given bbox of a given pixel resolution (in meters). The result is
#     rounded to the nearest integers.

#     :param bbox: bounding box
#     :param resolution: Resolution of desired image in meters. It can be a single number or a tuple of two numbers -
#         resolution in horizontal and resolution in vertical direction.
#     :return: width and height in pixels for given bounding box and pixel resolution

Image shape at 20 m resolution: (68, 39) pixels


##BBox to resolution
specify image size (in pixels) and get resolution of image

In [ ]:
betsiboka_size = bbox_to_resolution(bbox_coordinates, 137 ,79 ,True)

print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")

#  """Calculates pixel resolution for a given bbox of a given width and height. By default, it returns result in
#     meters.

#     :param bbox: bounding box
#     :param width: width of bounding box in pixels
#     :param height: height of bounding box in pixels
#     :param meters: If `True` result will be given in meters, otherwise it will be given in units of current CRS
#     :return: resolution east-west at north and south, and resolution north-south for given CRS
#     :raises: ValueError if CRS is not supported

Image shape at 10 m resolution: (9.977830713376042, 9.97329445701399) pixels


#Request data for Single Polygon using .kmz file

The code is for extracting

In [ ]:
def convert_rec(x):
    if isinstance(x, list):
        return list(map(convert_rec, x))
    else:
        return float(x)

In [ ]:
dir = '/content/6/'
print_once=True #print only a single loop
i=0
df=pd.DataFrame
count=0
for file in sorted(os.listdir(dir)):

  final_coordinates=[]

  if file.endswith(".kmz"):
    dir2 = os.path.join(dir,file)
    #print("Shwoing one directory path", dir2)

    ##unzipping each KMZ file leads to a doc.kml file
    kmz = ZipFile(dir2, 'r')
    kml = kmz.open('doc.kml', 'r').read()

    ##extract coordinated from each kml file
    root = parser.fromstring(kml)
    extracted_coordinates=root.Document.Placemark.Polygon.outerBoundaryIs.LinearRing.coordinates
    # print(extracted_coordinates)
    #print("Extracted Coordinates", dir2, extracted_coordinates)

    ##retrieve the coordinates as text from lxml.objectify.StringElement
    x=extracted_coordinates.text
    # print("Coordinates as Text", x)

    ##split coordinates by space
    splitted_coordinates = x.split()
    # print("Split coordinates by space", splitted_coordinates)

    ##create nested list
    newlist=[]
    for word in splitted_coordinates:
        word = word.split(",")
        newlist.append(word)
    # print("Nested list", newlist)

    ##convert string to int
    final_coordinates = convert_rec(newlist)
    # print(final_coordinates)

    ##create geometry
    Geometry1 = Geometry(geometry={"type":"Polygon","coordinates":[final_coordinates]}, crs=CRS.WGS84)

    ##specify time slots
    slots = wheat_time_interval(2021, 11, 1, 2022, 5, 15, 10) #start_year, start_month, start_date, end_year, end_month, end_date, chunks
    results = [slot[1] for slot in slots]
    print(results)

    #specify path for saving data in drive
    path = '/content/gdrive/MyDrive/Crop Mapping Data for Marking Wheat Region/'


    ##get and save data
    for slot in slots:
        data = get_true_color_request(slot, file, Geometry1, path)
        data.save_data()


Monthly time windows:

('2021-11-01', '2021-11-20')
('2021-11-20', '2021-12-10')
('2021-12-10', '2021-12-29')
('2021-12-29', '2022-01-18')
('2022-01-18', '2022-02-06')
('2022-02-06', '2022-02-26')
('2022-02-26', '2022-03-17')
('2022-03-17', '2022-04-06')
('2022-04-06', '2022-04-25')
['2021-11-20', '2021-12-10', '2021-12-29', '2022-01-18', '2022-02-06', '2022-02-26', '2022-03-17', '2022-04-06', '2022-04-25']


#Request data for Multiple Polygons in single image using .kml file

In [ ]:
# Create a new dataframe for storing the polygons
ground_truth = gpd.GeoDataFrame()
driver = 'KML'

#path to your .kml file
polygon_file_path = "/content/13.kml"
#extract filename from directory
directory, file = os.path.split(polygon_file_path)

# polpulate the dataframe using the .kml file containing all polygons
for layer in fiona.listlayers(polygon_file_path):
  s = gpd.read_file(polygon_file_path, driver=driver, layer=layer)
  # print(s.geometry)
  ground_truth = pd.concat([ground_truth, s], ignore_index=True)
  # print(ground_truth)

##extract geometrical coordinates of all the polygons in the .kml file
multi_polygon_coordinates=[]
shapefile = []
for geo_shape in ground_truth.geometry.values:
  # print(geo_shape)
  shapefile = geo_shape
  # new.extend([geo_shape])
  temp_polygon = list(map(list, (geo_shape.exterior.coords)))
  # print("temp_polygon", temp_polygon)
  multi_polygon_coordinates.append(list([temp_polygon]))

# ##create geometry
Geometry1 = Geometry(geometry={"type":"MultiPolygon","coordinates": multi_polygon_coordinates}, crs=CRS.WGS84)
# print(Geometry1)

##specify time slots
slots = wheat_time_interval(2021, 11, 1, 2022, 5, 15, 2) #start_year, start_month, start_date, end_year, end_month, end_date, chunks
results = [slot[1] for slot in slots]
print(results)

#specify path for saving data in drive
path = '/content/13'

##get and save data
for slot in slots:
  data = get_true_color_request(slot, file, Geometry1, path)
  data.save_data()

Monthly time windows:

('2021-11-01', '2022-02-06')
['2022-02-06']


In [ ]:
# Create a new dataframe for storing the polygons
ground_truth = gpd.GeoDataFrame()
driver = 'KML'

#path to your .kml file
polygon_file_path = "/content/Area 2 mask.kml"
#extract filename from directory
directory, file = os.path.split(polygon_file_path)

# polpulate the dataframe using the .kml file containing all polygons
for layer in fiona.listlayers(polygon_file_path):
  s = gpd.read_file(polygon_file_path, driver=driver, layer=layer)
  ground_truth = pd.concat([ground_truth, s], ignore_index=True)
  print(ground_truth)

##extract geometrical coordinates of all the polygons in the .kml file
multi_polygon_coordinates=[]
for geo_shape in ground_truth.geometry.values:
  # print(geo_shape)
  # new.extend([geo_shape])
  temp_polygon = list(map(list, (geo_shape.exterior.coords)))
  # print("temp_polygon", temp_polygon)
  multi_polygon_coordinates.append(list([temp_polygon]))
print(multi_polygon_coordinates)

##create geometry
Geometry1 = Geometry(geometry={"type":"MultiPolygon","coordinates": multi_polygon_coordinates}, crs=CRS.WGS84)

##specify time slots
slots = wheat_time_interval(2021, 11, 1, 2022, 5, 15, 2) #start_year, start_month, start_date, end_year, end_month, end_date, chunks
results = [slot[1] for slot in slots]
print(results)

#specify path for saving data in drive
path = '/content/gdrive/MyDrive/Crop Mapping/Sargodha Wheat 2022/'

##get and save data
for slot in slots:
  data = get_true_color_request(slot, file, Geometry1, path)
  data.save_data()


    Name Description                                           geometry
0  00023              POLYGON Z ((72.56754 32.30949 0.00000, 72.5688...
    Name Description                                           geometry
0  00023              POLYGON Z ((72.56754 32.30949 0.00000, 72.5688...
1  00024              POLYGON Z ((72.56781 32.31440 0.00000, 72.5677...
    Name Description                                           geometry
0  00023              POLYGON Z ((72.56754 32.30949 0.00000, 72.5688...
1  00024              POLYGON Z ((72.56781 32.31440 0.00000, 72.5677...
2  00025              POLYGON Z ((72.56781 32.31381 0.00000, 72.5676...
    Name Description                                           geometry
0  00023              POLYGON Z ((72.56754 32.30949 0.00000, 72.5688...
1  00024              POLYGON Z ((72.56781 32.31440 0.00000, 72.5677...
2  00025              POLYGON Z ((72.56781 32.31381 0.00000, 72.5676...
3  00027              POLYGON Z ((72.57003 32.31592 0.00000, 72.

#Rename files with time from .json file

In [ ]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:

#test->boundary->233h34h2jh232->response.tiff and request.json

#dir0 = '/content/gdrive/MyDrive/Crop Mapping/GitHub/Landsat-Classification-Using-Neural-Network/Gujranwala_9sqkm/'
# dir0 = '/content/gdrive/MyDrive/Crop Mapping/GitHub/Landsat-Classification-Using-Neural-Network/Gujranwala_9sqkm/Gujranwala_predict_builtup_1sqkm/'
# dir0 = '/content/gdrive/MyDrive/Phenology/may/'
dir0 = '/content/gdrive/MyDrive/Crop_Mapping/Final_Data/Sargodha_2021_1nov/'
for main_file in (os.listdir(dir0)):
  if main_file == "Sargodha.kml":
  # # #mainfile = boundary or 000000
  # # dir3 = '/content/gdrive/MyDrive/Phenology/2020-74/pol_20_7.kmz'
    dir3 = os.path.join(dir0, main_file)

    for fn in (os.listdir(dir3)):
      ##fn = timestamps i.e. 000000 22-10-22
      dir4 = os.path.join(dir3, dir3 + "/" + fn)
      json_file_name = ""
      tiff_file_name = ""
      for filename in os.listdir(dir4):
        ##filename = 22-10-22.tiff or json
        if os.path.join(dir4, filename).endswith(".json"):
          f = open(os.path.join(dir4, filename))
          data = json.load(f)
          for i in data:
            date= data['payload']['input']['data'][0]['dataFilter']['timeRange']['to']
            match = re.search(r'\d{4}-\d{2}-\d{2}', date)
            date = datetime.datetime.strptime(match.group(), '%Y-%m-%d').date()
            date = date.strftime('%Y-%m-%d')
            date_tiff = date +str(".tiff")
            date_json = date +str(".json")
            json_file_name = os.path.join(dir4, filename)
        elif os.path.join(dir4, filename).endswith(".tiff"):
          tiff_file_name = os.path.join(dir4, filename)
        if len(json_file_name) != 0 and len(tiff_file_name) != 0:
          os.rename(json_file_name, os.path.join(dir4, date_json))
          os.rename(tiff_file_name, os.path.join(dir4, date_tiff))

    os.rename(os.path.join(dir3,fn), os.path.join(dir3, date))




#Seperate .tiff and .json files

In [ ]:
import os
import shutil

source_folder = r"/content/gdrive/MyDrive/Crop_Mapping/Final_Data/Sargodha_2021_1nov/"
destination_folder_json = "/content/gdrive/MyDrive/Crop_Mapping/Final_Data/Sargodha_2021_1nov/Data.json/"
destination_folder_tiff = "/content/gdrive/MyDrive/Crop_Mapping/Final_Data/Sargodha_2021_1nov/Data.tiff/"

for main_file in (os.listdir(source_folder)):
  ##mainfile = boundary or 000000
    # print(main_file)
  # if (main_file) == "s2 data 30km2 updated.kml":
  dir3 = os.path.join(source_folder, main_file)

  for fn in (os.listdir(dir3)):
    ##fn = boundary/timestamps i.e. 000000 22-10-22
    dir4 = os.path.join(dir3, dir3 + "/" + fn)
    json_file_name = ""
    tiff_file_name = ""
    for filename in os.listdir(dir4):
      ##filename = boundary/timestamps/22-10-22.tiff or json
      if os.path.join(dir4, filename).endswith(".json"):
        json_file_name = os.path.join(dir4, filename)

      elif os.path.join(dir4, filename).endswith(".tiff"):
        tiff_file_name = os.path.join(dir4, filename)

      if len(json_file_name) != 0 and len(tiff_file_name) != 0:
        isExist_destination_folder_json = os.path.exists(destination_folder_json)
        isExist_destination_folder_tiff = os.path.exists(destination_folder_tiff)
        if not isExist_destination_folder_json:
          # Create a new directory because it does not exist
          os.makedirs(destination_folder_json)
        if not isExist_destination_folder_tiff:
          # Create a new directory because it does not exist
          os.makedirs(destination_folder_tiff)

        shutil.move(json_file_name, destination_folder_json)
        shutil.move(tiff_file_name, destination_folder_tiff)


#Cloud Removal

In [ ]:
def cloud_removal(i):

  #enumerate for the time-series of each patch (with all its bands)
  for ids, folder in enumerate(data_folders):

    # print(ids)                                                                  ##ids=0,1,2....
    print(folder)                                                               ##folder=  Data.json', 'Data.tiff', 'Mask'

    if (folder == "Data.tiff"):
      print(folder)
      n=0
      s2_data_path = os.path.join(data_path, folder)                            #/content/gdrive/MyDrive/Crop Mapping/Sentinel Hub Dataset for Code/1 S2_10m_all bands upsampled_5.5months_bboxtodimensions/Data.tiff
      # print(s2_data_path)
      s2_data = next(os.walk(s2_data_path))[2]
      # print(s2_data)                                                            #['2021-11-20.tiff', '2021-11-26.tiff'....

      for ids2, tiff_file in enumerate(s2_data):
        img = gdal.Open(os.path.join(s2_data_path, tiff_file) , gdal.GA_ReadOnly)                  #open each tiff file (with all its bands)
        # print(img)# img = img.ReadAsArray()                                 #read each tiff file as array (with all its bands)
        band = img.GetRasterBand(5)
        x_img_arr = band.ReadAsArray()
        print(x_img_arr.shape)                                 #read each tiff file as array (with all its bands)

        all_zeros = np.any(x_img_arr == 0) #o means no data, if any value is 0 then True
        print(all_zeros)
        if(all_zeros==True):
          if os.path.exists(os.path.join(s2_data_path, tiff_file)):
            os.remove(os.path.join(s2_data_path, tiff_file))
            n=n+1
            print(ids2)
      print("Images removed", n)


data_path = '/content/gdrive/MyDrive/Crop_Mapping/Final_Data/Sargodha_2021_1nov/'
data_folders = next(os.walk(data_path))[1]
# print(data_folders)                                                              ##data_folders=  Data.json', 'Data.tiff', 'Mask'
i=1
cloud_removal(i)                                                #create dataset with shape (time, height, width, bands)->(29, 79, 137, 12)

#164 total for sargodha, removed 132, remaining 32

Sargodha.kml
Data.json
Data.tiff
Data.tiff
(298, 212)
True
0
(298, 212)
False
(298, 212)
False
(298, 212)
False
(298, 212)
False
(298, 212)
False
(298, 212)
False
(298, 212)
False
(298, 212)
False
(298, 212)
False
(298, 212)
False
(298, 212)
True
11
(298, 212)
True
12
(298, 212)
True
13
(298, 212)
True
14
(298, 212)
True
15
(298, 212)
True
16
(298, 212)
True
17
(298, 212)
True
18
(298, 212)
False
(298, 212)
True
20
(298, 212)
False
(298, 212)
True
22
(298, 212)
True
23
(298, 212)
True
24
(298, 212)
True
25
(298, 212)
False
(298, 212)
False
(298, 212)
False
(298, 212)
False
(298, 212)
False
(298, 212)
False
Images removed 14
